# Data Preparation

## 0. Setup

In [1]:
%pip install genson

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import json
from pathlib import Path
from genson import SchemaBuilder
from typing import List

In [3]:
DATA_FOLDER_PATH = Path("../../data")

RAW_DATA_FOLDER_PATH = DATA_FOLDER_PATH.joinpath("raw/Data 2018-2023/Project")
RAW_DATA_SAMPLE_PATH = RAW_DATA_FOLDER_PATH.joinpath("2023/202302889")
RAW_DATA_SCHEMA_PATH = DATA_FOLDER_PATH.joinpath("schema/raw_data_schema.json")

PROCESSED_DATA_FOLDER_PATH = DATA_FOLDER_PATH.joinpath("processed")

## 1. Explore given data from Scopus

In [ ]:
with open(RAW_DATA_SAMPLE_PATH) as file:
    data = json.load(file)

In [5]:
print(json.dumps(data, indent=2))

{
  "abstracts-retrieval-response": {
    "item": {
      "ait:process-info": {
        "ait:status": {
          "@state": "update",
          "@type": "core",
          "@stage": "S300"
        },
        "ait:date-delivered": {
          "@day": "08",
          "@timestamp": "2023-06-08T15:03:15.000015-04:00",
          "@year": "2023",
          "@month": "06"
        },
        "ait:date-sort": {
          "@day": "01",
          "@year": "2023",
          "@month": "01"
        }
      },
      "xocs:meta": {
        "xocs:funding-list": {
          "@pui-match": "primary",
          "@has-funding-info": "1",
          "xocs:funding": {
            "xocs:funding-agency-matched-string": "Chulalongkorn University",
            "xocs:funding-agency-acronym": "CU",
            "xocs:funding-agency": "Chulalongkorn University",
            "xocs:funding-agency-id": "http://data.elsevier.com/vocabulary/SciValFunders/501100002873",
            "xocs:funding-agency-country": "http://sws.

## 2. Build schema

In [10]:
schema = {}

for path in RAW_DATA_FOLDER_PATH.glob("*/*"):
    print(path)
    with open(path.absolute()) as file:
        data = json.load(file)
    schema_builder = SchemaBuilder()
    schema_builder.add_schema(schema)
    schema_builder.add_object(data)
    schema = schema_builder.to_schema()

..\..\data\raw\Data 2018-2023\Project\2018\201800000
..\..\data\raw\Data 2018-2023\Project\2018\201800001
..\..\data\raw\Data 2018-2023\Project\2018\201800002
..\..\data\raw\Data 2018-2023\Project\2018\201800003
..\..\data\raw\Data 2018-2023\Project\2018\201800004
..\..\data\raw\Data 2018-2023\Project\2018\201800005
..\..\data\raw\Data 2018-2023\Project\2018\201800006
..\..\data\raw\Data 2018-2023\Project\2018\201800007
..\..\data\raw\Data 2018-2023\Project\2018\201800008
..\..\data\raw\Data 2018-2023\Project\2018\201800009
..\..\data\raw\Data 2018-2023\Project\2018\201800010
..\..\data\raw\Data 2018-2023\Project\2018\201800011
..\..\data\raw\Data 2018-2023\Project\2018\201800012
..\..\data\raw\Data 2018-2023\Project\2018\201800013
..\..\data\raw\Data 2018-2023\Project\2018\201800014
..\..\data\raw\Data 2018-2023\Project\2018\201800015
..\..\data\raw\Data 2018-2023\Project\2018\201800016
..\..\data\raw\Data 2018-2023\Project\2018\201800017
..\..\data\raw\Data 2018-2023\Project\2018\201

In [11]:
with open(RAW_DATA_SCHEMA_PATH, "w") as file:
    json.dump(schema, file, indent=2)

## 3. Explore fields

In [ ]:
def is_path_required(schema: dict, path: str | List[str]):
    if type(path) is not list:
        path = path.split(".")
    if len(path) == 0:
        return True

    if "items" in schema:
        return is_path_required(schema["items"], path)
    
    if "anyOf" in schema:
        for schema_part in schema["anyOf"]:
            if not is_path_required(schema_part, path):
                return False
        return True
    
    if "type" in schema and schema["type"] != "object":
        return False
    
    property = path[0]
    if property not in schema["required"]:
        return False
    else:
        schema_part = schema["properties"][property]
        if not is_path_required(schema_part, path[1:]):
            return False
    return True

In [13]:
with open(RAW_DATA_SAMPLE_PATH) as file:
    data = json.load(file)

In [14]:
with open(RAW_DATA_SCHEMA_PATH) as file:
    schema = json.load(file)

### 3.1 ID

In [ ]:
data["abstracts-retrieval-response"]["coredata"]["eid"]

'2-s2.0-85106046890'

In [90]:
is_path_required(schema, "abstracts-retrieval-response.coredata.eid")

True

### 3.2 Author

In [ ]:
pd.json_normalize(
    data,
    record_path=["abstracts-retrieval-response", "authors", "author"],
    errors="ignore",
)

,ce:given-name,@seq,ce:initials,@_fa,affiliation,ce:surname,@auid,author-url,ce:indexed-name,preferred-name.ce:given-name,preferred-name.ce:initials,preferred-name.ce:surname,preferred-name.ce:indexed-name
0,Vatana,1,V.,true,"[{'@id': '126229576', '@href': 'https://api.el...",Chea,57223009056,https://api.elsevier.com/content/author/author...,Chea V.,Vatana,V.,Chea,Chea V.


In [61]:
is_path_required(schema, "abstracts-retrieval-response.authors.author")

True

### 3.3 Article title

In [ ]:
data["abstracts-retrieval-response"]["coredata"]["dc:title"]

'Effects of remittances on household poverty and inequality in Cambodia'

In [76]:
is_path_required(schema, "abstracts-retrieval-response.coredata.dc:title")

False

### 3.4 Publication name
(background of the problem)

In [21]:
data["abstracts-retrieval-response"]["coredata"]["prism:publicationName"]

'Journal of the Asia Pacific Economy'

In [63]:
is_path_required(schema, "abstracts-retrieval-response.coredata.prism:publicationName")

True

### 3.5 Abstract

In [ ]:
print(data["abstracts-retrieval-response"]["item"]["bibrecord"]["head"]["abstracts"])

© 2021 Informa UK Limited, trading as Taylor & Francis Group.The author uses data from the Cambodia Socio-Economic Survey in 2014 to investigate the impact of remittances on poverty and inequality. Unlike other studies that use income to measure poverty, we employ monthly per capita consumption. We also consider remittances as a substitute income rather than an exogenous transfer. Therefore, imputing counterfactual expenditure in a scenario of no migration no remittances is necessary. To test for selection, a Heckman model is required under the null hypothesis that non-recipient households are randomly drawn from the population. Contrary to some previous studies, we find significant effect of selection bias and evidence that remittances reduce the poverty rate by 2 percent on the national level or 5 percent for recipient households. Furthermore, remittances decrease the poverty gap by 2.5 percent or 6.6 percent for a sub-sample of recipient households, but they also increase inequality

In [64]:
is_path_required(schema, "abstracts-retrieval-response.item.bibrecord.head.abstracts")

True

### 3.6 Classification codes

In [ ]:
# Classification codes (1) (Not used)
pd.json_normalize(
    data,
    record_path=["abstracts-retrieval-response", "item", "bibrecord", "head", "enhancement", "classificationgroup", "classifications"],
)

,@type,classification
0,GEOCLASS,"[{'classification-code': '70.12.6', 'classific..."
1,ASJC,"[{'$': '3305'}, {'$': '3303'}, {'$': '3320'}]"
2,SUBJABBR,SOCI


In [25]:
is_path_required(schema, "abstracts-retrieval-response.item.bibrecord.head.enhancement.classificationgroup.classifications")

True

In [ ]:
# Classification codes (2) (ASJC)
pd.json_normalize(
    data,
    record_path=["abstracts-retrieval-response", "subject-areas", "subject-area"],
)

,@_fa,$,@code,@abbrev
0,true,"Geography, Planning and Development",3305,SOCI
1,true,Development,3303,SOCI
2,true,Political Science and International Relations,3320,SOCI


In [65]:
is_path_required(schema, "abstracts-retrieval-response.subject-areas.subject-area")

True

### 3.7 Publication date

In [ ]:
# # Publication date (1)
# data["abstracts-retrieval-response"]["item"]["bibrecord"]["head"]["source"]["publicationdate"]

In [66]:
is_path_required(schema, "abstracts-retrieval-response.item.bibrecord.head.source.publicationdate")

True

In [ ]:
# Publication date (2)
data["abstracts-retrieval-response"]["coredata"]["prism:coverDate"]

'2023-01-01'

In [67]:
is_path_required(schema, "abstracts-retrieval-response.coredata.prism:coverDate")

True

### 3.8 Creation date

In [ ]:
# # Creation date
# data["abstracts-retrieval-response"]["item"]["bibrecord"]["item-info"]["history"]["date-created"]

In [68]:
is_path_required(schema, "abstracts-retrieval-response.item.bibrecord.item-info.history.date-created")

True

### 3.9 Affiliations

In [ ]:
# Affiliations (1)
affiliations_1 = pd.json_normalize(
    data,
    ["abstracts-retrieval-response", "item", "bibrecord", "head", "author-group"]
)

affiliations_1

,author,affiliation.country,affiliation.@afid,affiliation.@country,affiliation.city,affiliation.organization,affiliation.affiliation-id.@afid,affiliation.affiliation-id.@dptid,affiliation.@affiliation-instance-id,affiliation.ce:source-text,affiliation.@dptid
0,"[{'ce:given-name': 'Vatana', 'preferred-name':...",Cambodia,126229576,khm,Phnom Penh,[{'$': 'Faculty of Applied Science and Center ...,126229576,126231152,OB2BibRecID-933163792-3290dab63a1b75fafb6b1a09...,Faculty of Applied Science and Center for Prof...,126231152
1,"[{'ce:given-name': 'Vatana', 'preferred-name':...",Thailand,60028190,tha,Bangkok,[{'$': 'Formerly College of Population Studies...,60028190,104425769,OB2BibRecID-933163792-a85d06d37ec2dc1e74dbdab9...,"Formerly College of Population Studies, Chulal...",104425769


In [69]:
is_path_required(schema, "abstracts-retrieval-response.item.bibrecord.head.author-group")

True

In [ ]:
# Affiliations (2)
pd.json_normalize(
    data,
    ["abstracts-retrieval-response", "affiliation"]
)

,affiliation-city,@id,affilname,@href,affiliation-country
0,Bangkok,60028190,Chulalongkorn University,https://api.elsevier.com/content/affiliation/a...,Thailand
1,Phnom Penh,126229576,Cambodia University of Technology and Science,https://api.elsevier.com/content/affiliation/a...,Cambodia


In [70]:
is_path_required(schema, "abstracts-retrieval-response.affiliation")

True

### 3.10 Citation info

In [ ]:
# Citation info
data["abstracts-retrieval-response"]["item"]["bibrecord"]["head"]["citation-info"]

{'author-keywords': {'author-keyword': [{'$': 'Cambodia',
    '@xml:lang': 'eng',
    '@original': 'y'},
   {'$': 'consumption', '@xml:lang': 'eng', '@original': 'y'},
   {'$': 'expenditure', '@xml:lang': 'eng', '@original': 'y'},
   {'$': 'Impact', '@xml:lang': 'eng', '@original': 'y'},
   {'$': 'inequality', '@xml:lang': 'eng', '@original': 'y'},
   {'$': 'remittances', '@xml:lang': 'eng', '@original': 'y'}]},
 'citation-type': {'@code': 'ar'},
 'citation-language': {'@language': 'English', '@xml:lang': 'eng'},
 'abstract-language': {'@language': 'English', '@xml:lang': 'eng'}}

In [71]:
is_path_required(schema, "abstracts-retrieval-response.item.bibrecord.head.citation-info")

True

### 3.11 Cited by count

In [ ]:
# Cited by count
data["abstracts-retrieval-response"]["coredata"]["citedby-count"]

'3'

In [72]:
is_path_required(schema, "abstracts-retrieval-response.coredata.citedby-count")

True

### 3.12 Reference count

In [ ]:
# Reference count
data["abstracts-retrieval-response"]["item"]["bibrecord"]["tail"]["bibliography"]["@refcount"]

'58'

In [ ]:
is_path_required(schema, "abstracts-retrieval-response.item.bibrecord.tail.bibliography.@refcount")

False

### 3.13 References

In [ ]:
# References
references = pd.json_normalize(
    data,
    ["abstracts-retrieval-response", "item", "bibrecord", "tail", "bibliography", "reference"],
)
references["paper_id"] = data["abstracts-retrieval-response"]["coredata"]["eid"]
references.head(5)

,ref-fulltext,@reference-instance-id,@id,ce:source-text,ref-info.ref-publicationyear.@first,ref-info.ref-title.ref-titletext,ref-info.refd-itemidlist.itemid,ref-info.ref-volisspag.voliss.@volume,ref-info.ref-volisspag.voliss.@issue,ref-info.ref-volisspag.pagerange.@first,ref-info.ref-volisspag.pagerange.@last,ref-info.ref-authors.author,ref-info.ref-sourcetitle,ref-info.ref-text,ref-info.ref-website.ce:e-address.$,ref-info.ref-website.ce:e-address.@type,ref-info.ref-authors.collaboration.@seq,ref-info.ref-authors.collaboration.ce:text,ref-info.ref-authors.collaboration.ce:indexed-name,paper_id
0,"Acosta, P., C. Calderón, P. Fajnzylber, and H....",OB2BibRecID-933163792-0ef1e06438b563f7f91ddd54...,1,"Acosta, P., C. Calderón, P. Fajnzylber, and H....",2008,What is the Impact of International Remittance...,"[{'$': '10.1016/j.worlddev.2007.02.016', '@idt...",36,1,89,114,"[{'@seq': '1', 'ce:initials': 'P.', '@_fa': 't...",World Development,NaN,NaN,NaN,NaN,NaN,NaN,2-s2.0-85106046890
1,"Acosta, P., P. Fajnzylber, and H. Lopez. 2007....",OB2BibRecID-933163792-77cb73a183cad2dc4668d437...,2,"Acosta, P., P. Fajnzylber, and H. Lopez. 2007....",2007,The Impact of Remittances on Poverty and Human...,"[{'$': 'CIT0002', '@idtype': 'FRAGMENTID'}, {'...",NaN,NaN,59,98,"[{'@seq': '1', 'ce:initials': 'P.', '@_fa': 't...","International Migration, Economic Development ...","Özden C., Schiff M., (eds), The World Bank and...",NaN,NaN,NaN,NaN,NaN,2-s2.0-85106046890
2,"Adams, R. H. 1989. “ Worker Remittances and In...",OB2BibRecID-933163792-6e27c58ce5bcc8d4d877f626...,3,"Adams, R. H. 1989. “ Worker Remittances and In...",1989,Worker Remittances and Inequality in Rural Egypt,"[{'$': '10.1086/451775', '@idtype': 'DOI'}, {'...",38,1,45,71,"[{'@seq': '1', 'ce:initials': 'R.H.', '@_fa': ...",Economic Development and Cultural Change,NaN,http://www.jstor.org/stable/1154160,email,NaN,NaN,NaN,2-s2.0-85106046890
3,"Adams, R. H. 2006. “ Remittances, Poverty, and...",OB2BibRecID-933163792-bc393961b26fd35491f5b271...,4,"Adams, R. H. 2006. “ Remittances, Poverty, and...",2006,"Remittances, Poverty, and Investment in Guatemala","[{'$': 'CIT0004', '@idtype': 'FRAGMENTID'}, {'...",NaN,NaN,NaN,NaN,"[{'@seq': '1', 'ce:initials': 'R.H.', '@_fa': ...","International Migration, Remittances, and the ...","Özden C.̧., Schiff M., (eds), The World Bank a...",NaN,NaN,NaN,NaN,NaN,2-s2.0-85106046890
4,"Adams, R. H. 2011. “ Evaluating the Economic I...",OB2BibRecID-933163792-9d4a59374ba687e0e9e9d835...,5,"Adams, R. H. 2011. “ Evaluating the Economic I...",2011,Evaluating the Economic Impact of Internationa...,"[{'$': '10.1080/00220388.2011.563299', '@idtyp...",47,6,809,828,"[{'@seq': '1', 'ce:initials': 'R.H.', '@_fa': ...",Journal of Development Studies,NaN,NaN,NaN,NaN,NaN,NaN,2-s2.0-85106046890
5,"Adams, Richard H. 1991. “ The Economic Uses an...",OB2BibRecID-933163792-dc12ce5ecc46a8bad2fa32d3...,6,"Adams, Richard H. 1991. “ The Economic Uses an...",1991,The Economic Uses and Impact of International ...,"[{'$': '10.1086/451904', '@idtype': 'DOI'}, {'...",39,4,695,722,"[{'ce:given-name': 'Richard H.', '@seq': '1', ...",Economic Development and Cultural Change,NaN,NaN,NaN,NaN,NaN,NaN,2-s2.0-85106046890
6,"Adams, R. H., and A. Cuecuecha. 2010. “ Remitt...",OB2BibRecID-933163792-cb9d3f4c9a92d22e04a9432b...,7,"Adams, R. H., and A. Cuecuecha. 2010. “ Remitt...",2010,"Remittances, Household Expenditure and Investm...","[{'$': '10.1016/j.worlddev.2010.03.003', '@idt...",38,11,1626,1641,"[{'@seq': '1', 'ce:initials': 'R.H.', '@_fa': ...",World Development,NaN,NaN,NaN,NaN,NaN,NaN,2-s2.0-85106046890
7,"Adams, R. H., Jr, and J. Page. 2005. “ Do Inte...",OB2BibRecID-933163792-ce6c96a54ccd8b72d506066e...,8,"Adams, R. H., Jr, and J. Page. 2005. “ Do Inte...",2005,Do International Migration and Remittances Red...,"[{'$': '10.1016/j.worlddev.2005.05.004', '@idt...",33,10,1645,1669,"[{'@seq': '1', 'ce:initials': 'R.H.', '@_fa': ...",World Development,NaN,https://doi.org/http,email,NaN,NaN,NaN,2-s2

In [92]:
is_path_required(schema, "abstracts-retrieval-response.item.bibrecord.tail.bibliography.reference")

False

### 3.14 Keywords

In [ ]:
# Keywords (2)
keywords = pd.json_normalize(
    data,
    ["abstracts-retrieval-response", "authkeywords", "author-keyword"],
)
keywords = keywords[["$"]]
keywords

,$
0,Cambodia
1,consumption
2,expenditure
3,Impact
4,inequality
5,remittances


In [93]:
is_path_required(schema, "abstracts-retrieval-response.authkeywords.author-keyword")

False

## 4. Aggregate data

### 4.0 Setup

In [4]:
def get_from_path_or_none(data: dict, path: str | List[str]):
    if type(path) is not list:
        path = path.split(".")
    for property in path:
        if type(data) is not dict or property not in data:
            return None
        data = data[property]
    return data

In [5]:
def ensure_list_data(x):
    return x if isinstance(x, list) else [x]

### 4.1 Reading data

In [6]:
data_list = []
for path in RAW_DATA_FOLDER_PATH.glob("*/*"):
    try:
        print(path)
        with open(path) as file:
            data = json.load(file)
        filtered_data = {}
        filtered_data["id"] = get_from_path_or_none(data, ["abstracts-retrieval-response", "coredata", "eid"])
        filtered_data["title"] = get_from_path_or_none(data, ["abstracts-retrieval-response", "coredata", "dc:title"])
        filtered_data["publication_name"] = get_from_path_or_none(data, ["abstracts-retrieval-response", "coredata", "prism:publicationName"],)
        filtered_data["abstract"] = get_from_path_or_none(data, ["abstracts-retrieval-response", "item", "bibrecord", "head", "abstracts"])
        filtered_data["publish_date"] = get_from_path_or_none(data, ["abstracts-retrieval-response", "coredata", "prism:coverDate"])
        filtered_data["cited_by_count"] = get_from_path_or_none(data, ["abstracts-retrieval-response", "coredata", "citedby-count"])
        filtered_data["reference_count"] = get_from_path_or_none(data, ["abstracts-retrieval-response", "item", "bibrecord", "tail", "bibliography", "@refcount"])
        filtered_data["classification_codes"] = get_from_path_or_none(data, ["abstracts-retrieval-response", "subject-areas", "subject-area"])
        filtered_data["affiliations"] = get_from_path_or_none(data, ["abstracts-retrieval-response", "affiliation"])
        filtered_data["references"] = get_from_path_or_none(data, ["abstracts-retrieval-response", "item", "bibrecord", "tail", "bibliography", "reference"])
        filtered_data["keywords"] = get_from_path_or_none(data, ["abstracts-retrieval-response", "authkeywords", "author-keyword"])
        data_list.append(filtered_data)
    except Exception as e:
        print(f"Failed: {path}\nException: {repr(e)}")
print("Reading files done.")
merged_df = pd.DataFrame(data_list)
merged_df.head(5)

..\..\data\raw\Data 2018-2023\Project\2018\201800000
..\..\data\raw\Data 2018-2023\Project\2018\201800001
..\..\data\raw\Data 2018-2023\Project\2018\201800002
..\..\data\raw\Data 2018-2023\Project\2018\201800003
..\..\data\raw\Data 2018-2023\Project\2018\201800004
..\..\data\raw\Data 2018-2023\Project\2018\201800005
..\..\data\raw\Data 2018-2023\Project\2018\201800006
..\..\data\raw\Data 2018-2023\Project\2018\201800007
..\..\data\raw\Data 2018-2023\Project\2018\201800008
..\..\data\raw\Data 2018-2023\Project\2018\201800009
..\..\data\raw\Data 2018-2023\Project\2018\201800010
..\..\data\raw\Data 2018-2023\Project\2018\201800011
..\..\data\raw\Data 2018-2023\Project\2018\201800012
..\..\data\raw\Data 2018-2023\Project\2018\201800013
..\..\data\raw\Data 2018-2023\Project\2018\201800014
..\..\data\raw\Data 2018-2023\Project\2018\201800015
..\..\data\raw\Data 2018-2023\Project\2018\201800016
..\..\data\raw\Data 2018-2023\Project\2018\201800017
..\..\data\raw\Data 2018-2023\Project\2018\201

,id,title,publication_name,abstract,publish_date,cited_by_count,reference_count,classification_codes,affiliations,references,keywords
0,2-s2.0-85077976956,Public health and international epidemiology f...,"Radiology in Global Health: Strategies, Implem...",None,2018-12-31,1,76,"[{'@_fa': 'true', '$': 'Medicine (all)', '@cod...","[{'affiliation-city': 'Stanford', '@id': '6003...",[{'ref-fulltext': 'Winslow CE. The untilled fi...,None
1,2-s2.0-85060936020,Flexible Printed Active Antenna for Digital Te...,Progress in Electromagnetics Research Symposium,"© 2018 The Institute of Electronics, Informati...",2018-12-31,1,4,"[{'@_fa': 'true', '$': 'Electrical and Electro...","{'affiliation-city': 'Bangkok', '@id': '600281...","[{'ref-fulltext': 'Pratumsiri, T. And P. Janpu...",None
2,2-s2.0-85052201238,Parametric study of hydrogen production via so...,Chemical Engineering Science,© 2018 Elsevier LtdComputational fluid dynamic...,2018-12-31,21,42,"[{'@_fa': 'true', '$': 'Chemistry (all)', '@co...","{'affiliation-city': 'Bangkok', '@id': '600281...","[{'ref-fulltext': 'Abanades, J.C., Anthony, E....","[{'@_fa': 'true', '$': 'Circulating fluidized ..."
3,2-s2.0-85051498032,Superhydrophobic coating from fluoroalkylsilan...,Applied Surface Science,© 2018 Elsevier B.V. A superhydrophobic/supero...,2018-12-31,37,45,"[{'@_fa': 'true', '$': 'Chemistry (all)', '@co...","[{'affiliation-city': 'Hirosaki', '@id': '6003...","[{'ref-fulltext': 'Abadi, S.R.H., Sebzari, M.R...","[{'@_fa': 'true', '$': 'Encapsulation'}, {'@_f..."
4,2-s2.0-85050678366,Electrochemical impedance-based DNA sensor usi...,Analytica Chimica Acta,© 2018 Elsevier B.V. A label-free electrochemi...,2018-12-31,68,55,"[{'@_fa': 'true', '$': 'Analytical Chemistry',...","[{'affiliation-city': 'Bangkok', '@id': '60028...","[{'ref-fulltext': 'Davies, P.D.O., Pai, M., Th...","[{'@_fa': 'true', '$': 'acpcPNA'}, {'@_fa': 't..."


In [7]:
merged_df["publish_date"] = pd.to_datetime(merged_df["publish_date"])

In [8]:
list_columns = ["classification_codes", "affiliations", "references", "keywords"]
for list_column in list_columns:
    merged_df[list_column] = merged_df[list_column].apply(ensure_list_data)

In [9]:
json_columns = ["classification_codes", "affiliations", "references", "keywords"]
for json_column in json_columns:
    merged_df[json_column] = merged_df[json_column].apply(json.dumps)

In [10]:
merged_df.head(5)

,id,title,publication_name,abstract,publish_date,cited_by_count,reference_count,classification_codes,affiliations,references,keywords
0,2-s2.0-85077976956,Public health and international epidemiology f...,"Radiology in Global Health: Strategies, Implem...",None,2018-12-31,1,76,"[{""@_fa"": ""true"", ""$"": ""Medicine (all)"", ""@cod...","[{""affiliation-city"": ""Stanford"", ""@id"": ""6003...","[{""ref-fulltext"": ""Winslow CE. The untilled fi...",[null]
1,2-s2.0-85060936020,Flexible Printed Active Antenna for Digital Te...,Progress in Electromagnetics Research Symposium,"© 2018 The Institute of Electronics, Informati...",2018-12-31,1,4,"[{""@_fa"": ""true"", ""$"": ""Electrical and Electro...","[{""affiliation-city"": ""Bangkok"", ""@id"": ""60028...","[{""ref-fulltext"": ""Pratumsiri, T. And P. Janpu...",[null]
2,2-s2.0-85052201238,Parametric study of hydrogen production via so...,Chemical Engineering Science,© 2018 Elsevier LtdComputational fluid dynamic...,2018-12-31,21,42,"[{""@_fa"": ""true"", ""$"": ""Chemistry (all)"", ""@co...","[{""affiliation-city"": ""Bangkok"", ""@id"": ""60028...","[{""ref-fulltext"": ""Abanades, J.C., Anthony, E....","[{""@_fa"": ""true"", ""$"": ""Circulating fluidized ..."
3,2-s2.0-85051498032,Superhydrophobic coating from fluoroalkylsilan...,Applied Surface Science,© 2018 Elsevier B.V. A superhydrophobic/supero...,2018-12-31,37,45,"[{""@_fa"": ""true"", ""$"": ""Chemistry (all)"", ""@co...","[{""affiliation-city"": ""Hirosaki"", ""@id"": ""6003...","[{""ref-fulltext"": ""Abadi, S.R.H., Sebzari, M.R...","[{""@_fa"": ""true"", ""$"": ""Encapsulation""}, {""@_f..."
4,2-s2.0-85050678366,Electrochemical impedance-based DNA sensor usi...,Analytica Chimica Acta,© 2018 Elsevier B.V. A label-free electrochemi...,2018-12-31,68,55,"[{""@_fa"": ""true"", ""$"": ""Analytical Chemistry"",...","[{""affiliation-city"": ""Bangkok"", ""@id"": ""60028...","[{""ref-fulltext"": ""Davies, P.D.O., Pai, M., Th...","[{""@_fa"": ""true"", ""$"": ""acpcPNA""}, {""@_fa"": ""t..."


In [11]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20216 entries, 0 to 20215
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   id                    20216 non-null  object        
 1   title                 20215 non-null  object        
 2   publication_name      20216 non-null  object        
 3   abstract              19551 non-null  object        
 4   publish_date          20216 non-null  datetime64[ns]
 5   cited_by_count        20212 non-null  object        
 6   reference_count       19805 non-null  object        
 7   classification_codes  20216 non-null  object        
 8   affiliations          20216 non-null  object        
 9   references            20216 non-null  object        
 10  keywords              20216 non-null  object        
dtypes: datetime64[ns](1), object(10)
memory usage: 1.7+ MB


In [12]:
with open(PROCESSED_DATA_FOLDER_PATH.joinpath("merged.csv"), "w") as file:
    merged_df.to_csv(file, index=False)

## 5. Segregate data

### 5.0 Setup

In [4]:
merged_df = pd.read_csv(PROCESSED_DATA_FOLDER_PATH.joinpath("merged.csv"))
merged_df.head(5)

,id,title,publication_name,abstract,publish_date,cited_by_count,reference_count,classification_codes,affiliations,references,keywords
0,2-s2.0-85077976956,Public health and international epidemiology f...,"Radiology in Global Health: Strategies, Implem...",NaN,2018-12-31,1.0,76.0,"[{""@_fa"": ""true"", ""$"": ""Medicine (all)"", ""@cod...","[{""affiliation-city"": ""Stanford"", ""@id"": ""6003...","[{""ref-fulltext"": ""Winslow CE. The untilled fi...",[null]
1,2-s2.0-85060936020,Flexible Printed Active Antenna for Digital Te...,Progress in Electromagnetics Research Symposium,"© 2018 The Institute of Electronics, Informati...",2018-12-31,1.0,4.0,"[{""@_fa"": ""true"", ""$"": ""Electrical and Electro...","[{""affiliation-city"": ""Bangkok"", ""@id"": ""60028...","[{""ref-fulltext"": ""Pratumsiri, T. And P. Janpu...",[null]
2,2-s2.0-85052201238,Parametric study of hydrogen production via so...,Chemical Engineering Science,© 2018 Elsevier LtdComputational fluid dynamic...,2018-12-31,21.0,42.0,"[{""@_fa"": ""true"", ""$"": ""Chemistry (all)"", ""@co...","[{""affiliation-city"": ""Bangkok"", ""@id"": ""60028...","[{""ref-fulltext"": ""Abanades, J.C., Anthony, E....","[{""@_fa"": ""true"", ""$"": ""Circulating fluidized ..."
3,2-s2.0-85051498032,Superhydrophobic coating from fluoroalkylsilan...,Applied Surface Science,© 2018 Elsevier B.V. A superhydrophobic/supero...,2018-12-31,37.0,45.0,"[{""@_fa"": ""true"", ""$"": ""Chemistry (all)"", ""@co...","[{""affiliation-city"": ""Hirosaki"", ""@id"": ""6003...","[{""ref-fulltext"": ""Abadi, S.R.H., Sebzari, M.R...","[{""@_fa"": ""true"", ""$"": ""Encapsulation""}, {""@_f..."
4,2-s2.0-85050678366,Electrochemical impedance-based DNA sensor usi...,Analytica Chimica Acta,© 2018 Elsevier B.V. A label-free electrochemi...,2018-12-31,68.0,55.0,"[{""@_fa"": ""true"", ""$"": ""Analytical Chemistry"",...","[{""affiliation-city"": ""Bangkok"", ""@id"": ""60028...","[{""ref-fulltext"": ""Davies, P.D.O., Pai, M., Th...","[{""@_fa"": ""true"", ""$"": ""acpcPNA""}, {""@_fa"": ""t..."


In [5]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20216 entries, 0 to 20215
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    20216 non-null  object 
 1   title                 20215 non-null  object 
 2   publication_name      20216 non-null  object 
 3   abstract              19551 non-null  object 
 4   publish_date          20216 non-null  object 
 5   cited_by_count        20212 non-null  float64
 6   reference_count       19805 non-null  float64
 7   classification_codes  20216 non-null  object 
 8   affiliations          20216 non-null  object 
 9   references            20216 non-null  object 
 10  keywords              20216 non-null  object 
dtypes: float64(2), object(9)
memory usage: 1.7+ MB


In [6]:
merged_df = merged_df.astype({
    "cited_by_count": pd.Int64Dtype(),
    "reference_count": pd.Int64Dtype(),
})

In [7]:
list_columns = ["classification_codes", "affiliations", "references", "keywords"]
for list_column in list_columns:
    merged_df[list_column] = merged_df[list_column].apply(json.loads)

### 5.1 Papers

In [9]:
selected_columns = ["id", "title", "publication_name", "abstract", "publish_date", "cited_by_count", "reference_count"]
papers_df = merged_df[selected_columns]
papers_df.head(5)

,id,title,publication_name,abstract,publish_date,cited_by_count,reference_count
0,2-s2.0-85077976956,Public health and international epidemiology f...,"Radiology in Global Health: Strategies, Implem...",NaN,2018-12-31,1,76
1,2-s2.0-85060936020,Flexible Printed Active Antenna for Digital Te...,Progress in Electromagnetics Research Symposium,"© 2018 The Institute of Electronics, Informati...",2018-12-31,1,4
2,2-s2.0-85052201238,Parametric study of hydrogen production via so...,Chemical Engineering Science,© 2018 Elsevier LtdComputational fluid dynamic...,2018-12-31,21,42
3,2-s2.0-85051498032,Superhydrophobic coating from fluoroalkylsilan...,Applied Surface Science,© 2018 Elsevier B.V. A superhydrophobic/supero...,2018-12-31,37,45
4,2-s2.0-85050678366,Electrochemical impedance-based DNA sensor usi...,Analytica Chimica Acta,© 2018 Elsevier B.V. A label-free electrochemi...,2018-12-31,68,55


In [10]:
papers_df["id"].is_unique

True

In [11]:
with open(PROCESSED_DATA_FOLDER_PATH.joinpath("papers.csv"), "w") as file:
    papers_df.to_csv(file, index=False)

### 5.2 Classification codes

In [18]:
paper_to_classification_code_df = merged_df[["id", "classification_codes"]].explode("classification_codes", ignore_index=True)
paper_to_classification_code_df.dropna(subset=["classification_codes"], inplace=True)
paper_to_classification_code_df.head(5)

,id,classification_codes
0,2-s2.0-85077976956,"{'@_fa': 'true', '$': 'Medicine (all)', '@code..."
1,2-s2.0-85060936020,"{'@_fa': 'true', '$': 'Electrical and Electron..."
2,2-s2.0-85060936020,"{'@_fa': 'true', '$': 'Electronic, Optical and..."
3,2-s2.0-85052201238,"{'@_fa': 'true', '$': 'Chemistry (all)', '@cod..."
4,2-s2.0-85052201238,"{'@_fa': 'true', '$': 'Chemical Engineering (a..."


In [19]:
classification_codes_df = pd.json_normalize(paper_to_classification_code_df["classification_codes"])
classification_codes_df.head(5)

,@_fa,$,@code,@abbrev
0,true,Medicine (all),2700,MEDI
1,true,Electrical and Electronic Engineering,2208,ENGI
2,true,"Electronic, Optical and Magnetic Materials",2504,MATE
3,true,Chemistry (all),1600,CHEM
4,true,Chemical Engineering (all),1500,CENG


In [20]:
classification_codes_df.drop(columns=["@_fa"], inplace=True)
classification_codes_df.rename(columns={
    "$": "name",
    "@code": "code",
    "@abbrev": "abbreviation"
}, inplace=True)
classification_codes_df.head(5)

,name,code,abbreviation
0,Medicine (all),2700,MEDI
1,Electrical and Electronic Engineering,2208,ENGI
2,"Electronic, Optical and Magnetic Materials",2504,MATE
3,Chemistry (all),1600,CHEM
4,Chemical Engineering (all),1500,CENG


In [21]:
paper_to_classification_code_df.rename(columns={"id": "paper_id"}, inplace=True)
paper_to_classification_code_df = paper_to_classification_code_df[["paper_id"]].join(classification_codes_df["code"], how="left")
paper_to_classification_code_df.head(5)

,paper_id,code
0,2-s2.0-85077976956,2700
1,2-s2.0-85060936020,2208
2,2-s2.0-85060936020,2504
3,2-s2.0-85052201238,1600
4,2-s2.0-85052201238,1500


In [22]:
paper_to_classification_code_df.drop_duplicates(ignore_index=True, inplace=True)

In [23]:
classification_codes_df.drop_duplicates(subset=["code"], ignore_index=True, inplace=True)

In [24]:
with open(PROCESSED_DATA_FOLDER_PATH.joinpath("classification_codes.csv"), "w") as file:
    classification_codes_df.to_csv(file, index=False)

In [25]:
with open(PROCESSED_DATA_FOLDER_PATH.joinpath("paper_to_classification_code.csv"), "w") as file:
    paper_to_classification_code_df.to_csv(file, index=False)

### 5.3 Affiliations

In [26]:
paper_to_affiliation_df = merged_df[["id", "affiliations"]].explode("affiliations", ignore_index=True)
paper_to_affiliation_df.dropna(subset=["affiliations"], inplace=True)
paper_to_affiliation_df.head(5)

,id,affiliations
0,2-s2.0-85077976956,"{'affiliation-city': 'Stanford', '@id': '60032..."
1,2-s2.0-85077976956,"{'affiliation-city': 'Bangkok', '@id': '600281..."
2,2-s2.0-85077976956,"{'affiliation-city': 'Bangkok', '@id': '600205..."
3,2-s2.0-85077976956,"{'affiliation-city': 'Stanford', '@id': '60015..."
4,2-s2.0-85077976956,"{'affiliation-city': 'Palo Alto', '@id': '6001..."


In [27]:
affiliations_df = pd.json_normalize(paper_to_affiliation_df["affiliations"])
paper_to_affiliation_df.drop(columns=["affiliations"], inplace=True)
affiliations_df.head(5)

,affiliation-city,@id,affilname,@href,affiliation-country
0,Stanford,60032838,Stanford University School of Medicine,https://api.elsevier.com/content/affiliation/a...,United States
1,Bangkok,60028190,Chulalongkorn University,https://api.elsevier.com/content/affiliation/a...,Thailand
2,Bangkok,60020543,Bumrungrad International Hospital,https://api.elsevier.com/content/affiliation/a...,Thailand
3,Stanford,60015615,Stanford Healthcare,https://api.elsevier.com/content/affiliation/a...,United States
4,Palo Alto,60012708,Stanford University,https://api.elsevier.com/content/affiliation/a...,United States


In [28]:
affiliations_df.rename(columns={
    "@id": "id",
    "affiliation-city": "city",
    "affilname": "name",
    "@href": "href",
    "affiliation-country": "country",
}, inplace=True)
affiliations_df.head(5)

,city,id,name,href,country
0,Stanford,60032838,Stanford University School of Medicine,https://api.elsevier.com/content/affiliation/a...,United States
1,Bangkok,60028190,Chulalongkorn University,https://api.elsevier.com/content/affiliation/a...,Thailand
2,Bangkok,60020543,Bumrungrad International Hospital,https://api.elsevier.com/content/affiliation/a...,Thailand
3,Stanford,60015615,Stanford Healthcare,https://api.elsevier.com/content/affiliation/a...,United States
4,Palo Alto,60012708,Stanford University,https://api.elsevier.com/content/affiliation/a...,United States


In [29]:
paper_to_affiliation_df.rename(columns={"id": "paper_id"}, inplace=True)
paper_to_affiliation_df = paper_to_affiliation_df.join(affiliations_df["id"], how="left")
paper_to_affiliation_df.head(5)

,paper_id,id
0,2-s2.0-85077976956,60032838
1,2-s2.0-85077976956,60028190
2,2-s2.0-85077976956,60020543
3,2-s2.0-85077976956,60015615
4,2-s2.0-85077976956,60012708


In [30]:
paper_to_affiliation_df.drop_duplicates(ignore_index=True, inplace=True)

In [31]:
affiliations_df.drop_duplicates(subset=["id"], ignore_index=True, inplace=True)

In [32]:
with open(PROCESSED_DATA_FOLDER_PATH.joinpath("affiliations.csv"), "w") as file:
    affiliations_df.to_csv(file, index=False)

In [33]:
with open(PROCESSED_DATA_FOLDER_PATH.joinpath("paper_to_affiliation.csv"), "w") as file:
    paper_to_affiliation_df.to_csv(file, index=False)

### 5.4 References

In [34]:
references_df = merged_df[["id", "references"]].explode("references", ignore_index=True)
references_df.dropna(subset=["references"], inplace=True)
references_df.head(5)

,id,references
0,2-s2.0-85077976956,{'ref-fulltext': 'Winslow CE. The untilled fie...
1,2-s2.0-85077976956,{'ref-fulltext': 'Institute of Medicine (U.S.)...
2,2-s2.0-85077976956,"{'ref-fulltext': 'Heggenhougen K, Quah SR. Int..."
3,2-s2.0-85077976956,{'ref-fulltext': 'Kirch W. Encyclopedia of pub...
4,2-s2.0-85077976956,{'ref-fulltext': 'Definition of Public health ...


In [35]:
references_df = references_df[["id"]].join(pd.json_normalize(references_df["references"]), how="left")
references_df.head(5)

,id,ref-fulltext,@id,ref-info.ref-publicationyear.@first,ref-info.ref-title.ref-titletext,ref-info.refd-itemidlist.itemid.$,ref-info.refd-itemidlist.itemid.@idtype,ref-info.ref-volisspag.voliss.@volume,ref-info.ref-volisspag.pagerange.@first,ref-info.ref-volisspag.pagerange.@last,...,ref-info.refd-itemidlist.itemid,ref-info.ref-website.websitename,ref-info.ref-volisspag.pagecount.$,ref-info.ref-volisspag.pagecount.@type,ce:source-text,@aii:was-generated-by,@reference-instance-id,ref-info.ref-publicationyear.@last,@date-locked,ref-info.ref-authors.collaboration
0,2-s2.0-85077976956,Winslow CE. The untilled fields of public heal...,1,1920,The untilled fields of public health,0002667983,SGR,51,23,33,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2-s2.0-85077976956,Institute of Medicine (U.S.). Committee for th...,2,1988,Committee for the Study of the Future of Publi...,33750367977,SGR,12,225,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2-s2.0-85077976956,"Heggenhougen K, Quah SR. International encyclo...",3,2008,NaN,85013970385,SGR,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2-s2.0-85077976956,Kirch W. Encyclopedia of public health. New Yo...,4,2008,NaN,77953026614,SGR,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2-s2.0-85077976956,Definition of Public health 2010 [cited 2010 D...,5,2010,NaN,85078067388,SGR,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
references_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 907787 entries, 0 to 908197
Data columns (total 31 columns):
 #   Column                                              Non-Null Count   Dtype  
---  ------                                              --------------   -----  
 0   id                                                  907787 non-null  object 
 1   ref-fulltext                                        907373 non-null  object 
 2   @id                                                 907376 non-null  object 
 3   ref-info.ref-publicationyear.@first                 890653 non-null  object 
 4   ref-info.ref-title.ref-titletext                    791357 non-null  object 
 5   ref-info.refd-itemidlist.itemid.$                   302431 non-null  object 
 6   ref-info.refd-itemidlist.itemid.@idtype             302431 non-null  object 
 7   ref-info.ref-volisspag.voliss.@volume               782267 non-null  object 
 8   ref-info.ref-volisspag.pagerange.@first             766155 non-null  

In [37]:
references_df[references_df.columns[:15]].head(5)

,id,ref-fulltext,@id,ref-info.ref-publicationyear.@first,ref-info.ref-title.ref-titletext,ref-info.refd-itemidlist.itemid.$,ref-info.refd-itemidlist.itemid.@idtype,ref-info.ref-volisspag.voliss.@volume,ref-info.ref-volisspag.pagerange.@first,ref-info.ref-volisspag.pagerange.@last,ref-info.ref-authors.author,ref-info.ref-sourcetitle,ref-info.ref-text,ref-info.ref-authors.collaboration.@seq,ref-info.ref-authors.collaboration.ce:text
0,2-s2.0-85077976956,Winslow CE. The untilled fields of public heal...,1,1920,The untilled fields of public health,0002667983,SGR,51,23,33,"[{'@seq': '1', 'ce:initials': 'C.E.', '@_fa': ...",Science.,NaN,NaN,NaN
1,2-s2.0-85077976956,Institute of Medicine (U.S.). Committee for th...,2,1988,Committee for the Study of the Future of Publi...,33750367977,SGR,12,225,NaN,NaN,The future of public health,"Washington, D.C: National Academy Press",1,Institute of Medicine (U.S.)
2,2-s2.0-85077976956,"Heggenhougen K, Quah SR. International encyclo...",3,2008,NaN,85013970385,SGR,NaN,NaN,NaN,"[{'@seq': '1', 'ce:initials': 'K.', '@_fa': 't...",International encyclopedia of public health.,Oxford: Academic,NaN,NaN
3,2-s2.0-85077976956,Kirch W. Encyclopedia of public health. New Yo...,4,2008,NaN,77953026614,SGR,NaN,NaN,NaN,"[{'@seq': '1', 'ce:initials': 'W.', '@_fa': 't...",Encyclopedia of public health.,New York:Springer,NaN,NaN
4,2-s2.0-85077976956,Definition of Public health 2010 [cited 2010 D...,5,2010,NaN,85078067388,SGR,NaN,NaN,NaN,NaN,Definition of Public health,cited 2010 December 08]. Available from,NaN,NaN


In [38]:
references_df[references_df.columns[15:]].head(5)

,ref-info.ref-authors.collaboration.ce:indexed-name,ref-info.ref-website.ce:e-address.$,ref-info.ref-website.ce:e-address.@type,ref-info.ref-authors.et-al,ref-info.ref-volisspag.voliss.@issue,ref-info.ref-volisspag.pages,ref-info.refd-itemidlist.itemid,ref-info.ref-website.websitename,ref-info.ref-volisspag.pagecount.$,ref-info.ref-volisspag.pagecount.@type,ce:source-text,@aii:was-generated-by,@reference-instance-id,ref-info.ref-publicationyear.@last,@date-locked,ref-info.ref-authors.collaboration
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Institute of Medicine (U.S.),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,http://www.medterms.com/script/main/art.asp?ar...,email,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
references_df = references_df[[
    "id", "@id", "ref-fulltext", "ref-info.ref-title.ref-titletext", "ref-info.ref-authors.author",
    "ref-info.ref-sourcetitle", "ref-info.ref-text"
]]
references_df.rename(columns={
    "id": "paper_id",
    "@id": "reference_id",
    "ref-fulltext": "full_text",
    "ref-info.ref-title.ref-titletext": "title",
    "ref-info.ref-authors.author": "authors",
    "ref-info.ref-sourcetitle": "source_title",
    "ref-info.ref-text": "text",
}, inplace=True)
references_df.head(5)

,paper_id,reference_id,full_text,title,authors,source_title,text
0,2-s2.0-85077976956,1,Winslow CE. The untilled fields of public heal...,The untilled fields of public health,"[{'@seq': '1', 'ce:initials': 'C.E.', '@_fa': ...",Science.,NaN
1,2-s2.0-85077976956,2,Institute of Medicine (U.S.). Committee for th...,Committee for the Study of the Future of Publi...,NaN,The future of public health,"Washington, D.C: National Academy Press"
2,2-s2.0-85077976956,3,"Heggenhougen K, Quah SR. International encyclo...",NaN,"[{'@seq': '1', 'ce:initials': 'K.', '@_fa': 't...",International encyclopedia of public health.,Oxford: Academic
3,2-s2.0-85077976956,4,Kirch W. Encyclopedia of public health. New Yo...,NaN,"[{'@seq': '1', 'ce:initials': 'W.', '@_fa': 't...",Encyclopedia of public health.,New York:Springer
4,2-s2.0-85077976956,5,Definition of Public health 2010 [cited 2010 D...,NaN,NaN,Definition of Public health,cited 2010 December 08]. Available from


#### 5.4.1 Authors

In [40]:
paper_reference_author_df = references_df[["paper_id", "reference_id", "authors"]].explode("authors", ignore_index=True)
paper_reference_author_df.dropna(subset=["authors"], inplace=True)
paper_reference_author_df.head(5)

,paper_id,reference_id,authors
0,2-s2.0-85077976956,1,"{'@seq': '1', 'ce:initials': 'C.E.', '@_fa': '..."
2,2-s2.0-85077976956,3,"{'@seq': '1', 'ce:initials': 'K.', '@_fa': 'tr..."
3,2-s2.0-85077976956,3,"{'@seq': '2', 'ce:initials': 'S.R.', '@_fa': '..."
4,2-s2.0-85077976956,4,"{'@seq': '1', 'ce:initials': 'W.', '@_fa': 'tr..."
8,2-s2.0-85077976956,8,"{'@seq': '1', 'ce:initials': 'L.', '@_fa': 'tr..."


In [41]:
paper_reference_author_df = paper_reference_author_df[["paper_id", "reference_id"]].join(pd.json_normalize(paper_reference_author_df["authors"]), how="left")
paper_reference_author_df.head(5)

,paper_id,reference_id,@seq,ce:initials,@_fa,ce:surname,ce:indexed-name,ce:suffix,ce:given-name,ce:degrees,ce:alias
0,2-s2.0-85077976956,1,1,C.E.,true,Winslow,Winslow C.E.,NaN,NaN,NaN,NaN
2,2-s2.0-85077976956,3,2,S.R.,true,Quah,Quah S.R.,NaN,NaN,NaN,NaN
3,2-s2.0-85077976956,3,1,W.,true,Kirch,Kirch W.,NaN,NaN,NaN,NaN
4,2-s2.0-85077976956,4,1,L.,true,Breslow,Breslow L.,NaN,NaN,NaN,NaN
8,2-s2.0-85077976956,8,1,B.J.,true,Hillman,Hillman B.J.,NaN,NaN,NaN,NaN


In [42]:
paper_reference_author_df = paper_reference_author_df[["paper_id", "reference_id", "ce:indexed-name"]]
paper_reference_author_df.rename(columns={"ce:indexed-name": "name"}, inplace=True)
paper_reference_author_df.head(5)

,paper_id,reference_id,name
0,2-s2.0-85077976956,1,Winslow C.E.
2,2-s2.0-85077976956,3,Quah S.R.
3,2-s2.0-85077976956,3,Kirch W.
4,2-s2.0-85077976956,4,Breslow L.
8,2-s2.0-85077976956,8,Hillman B.J.


In [43]:
references_df.drop(columns=["authors"], inplace=True)
references_df.head(5)

,paper_id,reference_id,full_text,title,source_title,text
0,2-s2.0-85077976956,1,Winslow CE. The untilled fields of public heal...,The untilled fields of public health,Science.,NaN
1,2-s2.0-85077976956,2,Institute of Medicine (U.S.). Committee for th...,Committee for the Study of the Future of Publi...,The future of public health,"Washington, D.C: National Academy Press"
2,2-s2.0-85077976956,3,"Heggenhougen K, Quah SR. International encyclo...",NaN,International encyclopedia of public health.,Oxford: Academic
3,2-s2.0-85077976956,4,Kirch W. Encyclopedia of public health. New Yo...,NaN,Encyclopedia of public health.,New York:Springer
4,2-s2.0-85077976956,5,Definition of Public health 2010 [cited 2010 D...,NaN,Definition of Public health,cited 2010 December 08]. Available from


In [44]:
with open(PROCESSED_DATA_FOLDER_PATH.joinpath("references.csv"), "w") as file:
    references_df.to_csv(file, index=False)

In [45]:
with open(PROCESSED_DATA_FOLDER_PATH.joinpath("paper_reference_author.csv"), "w") as file:
    paper_reference_author_df.to_csv(file, index=False)

### 5.5 Keywords

In [48]:
paper_to_keyword_df = merged_df[['id', 'keywords']].explode('keywords', ignore_index=True)
paper_to_keyword_df.dropna(subset=['keywords'], inplace=True)
paper_to_keyword_df.head(5)

,id,keywords
2,2-s2.0-85052201238,"{'@_fa': 'true', '$': 'Circulating fluidized b..."
3,2-s2.0-85052201238,"{'@_fa': 'true', '$': 'Computational fluid dyn..."
4,2-s2.0-85052201238,"{'@_fa': 'true', '$': 'Multiphase flow models'}"
5,2-s2.0-85052201238,"{'@_fa': 'true', '$': 'Riser'}"
6,2-s2.0-85052201238,"{'@_fa': 'true', '$': 'Sorption enhanced steam..."


In [49]:
paper_to_keyword_df = paper_to_keyword_df[['id']].join(pd.json_normalize(paper_to_keyword_df['keywords']), how='left')
paper_to_keyword_df.head(5)

,id,@_fa,$
2,2-s2.0-85052201238,true,Multiphase flow models
3,2-s2.0-85052201238,true,Riser
4,2-s2.0-85052201238,true,Sorption enhanced steam methane reforming
5,2-s2.0-85052201238,true,Encapsulation
6,2-s2.0-85052201238,true,Fluoroalkylsilane
